This is suppose to be a quick (1 afternoon) application of a likelihood method to measure the effective popualtion of flu. We don't fix our neutrality assumption but it's a start. Here we don't assume a large Ne. So that's good.

If this goes forward I'll use the distribution of frequency from the benchmarking experiment for each titer, but for now I'll use the $10^4$ distribution. A normal distribution $\mu=0$, $\sigma^2=0.0002$, $\sigma = 0.014$ 

Addapting the set up in Williamsom and Slatkin we are after

$$
P(p_0,p_t|N_e) = \sum_{q_0,q_t}P(p_0|q_0)P(q_0|N_e)P(p_t|q_t)P(q_t|q_0,N_e)
$$

Where p are the observed probabilities and q are the real ones.

$$
P(p_x|q_x) = N(q_x,0.014)
$$

$$
P(q_0|N_e) = \frac{1}{N_e-1}
$$ 
as we are only looking at polymorphic sites

and 

$$
P(q_t|q_0,N_e) = v_0M^tv_t
$$

Where M is a square transmisssion matrix with $C=N+1$ rows and columns. Where $m_{i,j}$ is the probablilty of going from the ith configuration to the jth or the probability of drawing j out of binomial distirubution with mean $i/N_e$ and sample size $N_e$ and $v_0$ is a row vector of initial frequencies $q_0$ and $v_t$ is column vector of the frequencies at time point $t$ with 100% chance of the observed frequencies i.e. ( zeroes everywhere except the ith component =1 where $i/N_e = q_0$



### Modifying 

The summation terms are computationaly intense. I noticed that for a given q_0i and q_tj the term is 
$$
P(p_0|q_{0i})P(q_{0i}|N_e)[0 0 ... 1_i ... 0]M^t\begin{bmatrix}
    0 \\
    0 \\
    \vdots \\
    1_j \\
    \vdots \\
    0
\end{bmatrix} P(p_t|q_{tj})
$$


This can be simplified to 

$$
[0 0 ... P(p_0|q_{0i})P(q_{0i}|N_e) ... 0]M^t\begin{bmatrix}
    0 \\
    0 \\
    \vdots \\
    P(p_t|q_{tj}) \\
    \vdots \\
    0
\end{bmatrix} 
$$

Summing accross all i and j yeilds
$$
P(p_0,p_t|N_e) = [0, P(p_0|q_{02})P(q_{02}|N_e),  ...,  P(p_0|q_{0Ne-1})P(q_{0_Ne-1}|N_e), 0 ]M^t \begin{bmatrix}
    P(p_t|q_{t1}) \\
    \vdots \\
    P(p_t|q_{tNe})
\end{bmatrix} 
$$

Which is much more computationaly feasible. Linear algebra to the rescue!


In [1]:
addprocs(8)
@everywhere using Distributions
@everywhere  using DataFrames
#@everywhere using Plots; pyplot()
#@everywhere using ProgressMeter
@everywhere include("../scripts/Slatkin_functions.jl")

In [2]:
#if !Plots.is_installed("PyPlot")
#    Pkg.add("PyPlot")
#end
#withenv("PYTHON" => "") do
#    Pkg.build("PyPlot")
#end

In [3]:
function write_to_summary(line_pattern,value)
    lines = []    
    sr = Regex("^$line_pattern.*")
    open("./results.table.tsv") do results
        for line in enumerate(eachline(results))
            line_s = line[2]
            if ismatch(sr,line_s)
            line_s = "$line_pattern\t$value\n"
            end
        push!(lines,line_s)
        end
    end
    open("./results.table.tsv","w") do output
        for l in lines
           write(output,"$l")
        end
    end
end



write_to_summary (generic function with 1 method)

## Test run

Here is the Density (likelihood?) for a variant starting at 15% in a population of 100 1 generation later.

In [4]:
pₓ = linspace(0.0,1.0,100)

prob = zeros(length(pₓ))

p₀ = 0.15
Nₑ = 100
t = 1
for i in 1:length(pₓ)
    prob[i] = pTransition(pₓ[i],p₀,Nₑ,t)
end

prob
#plot(pₓ,prob)

100-element Array{Float64,1}:
 0.00139441 
 0.00582521 
 0.020433   
 0.0613627  
 0.160338   
 0.369471   
 0.759378   
 1.40553    
 2.36206    
 3.62972    
 5.13145    
 6.70977    
 8.1528     
 ⋮          
 1.16008e-45
 6.31345e-47
 3.23042e-48
 1.54976e-49
 6.94952e-51
 2.90307e-52
 1.12545e-53
 4.03198e-55
 1.32848e-56
 4.00389e-58
 1.09702e-59
 2.71317e-61

## Now 10 generations later

In [5]:
p₀ = 0.15
Nₑ = 100
t = 10
for i in 1:length(pₓ)
    prob[i] = pTransition(pₓ[i],p₀,Nₑ,t)
end

prob
#plot(pₓ,prob)

100-element Array{Float64,1}:
 2.82246   
 3.26313   
 3.26439   
 3.27785   
 3.39557   
 3.52804   
 3.62991   
 3.69604   
 3.72922   
 3.73248   
 3.70868   
 3.66064   
 3.59112   
 ⋮         
 5.90748e-6
 3.78004e-6
 2.36602e-6
 1.4447e-6 
 8.57647e-7
 4.92928e-7
 2.72816e-7
 1.44382e-7
 7.23763e-8
 3.39127e-8
 1.45654e-8
 5.56041e-9

# The real data

Here we read in the real data. We will only be using data points that were taken at least 1 day apart. Also we assume a generation time of 6 hrs or 4 generations in a day

In [6]:
data=readtable("../data/processed/secondary/Intrahost_initially_present.csv")
minor = data[data[:freq1].<=0.5,:]
minor = minor[minor[:within_host_time].>0,:]
minor[:generations] = minor[:within_host_time]*4
minor

,x,chr,pos,SPECID1,SPECID2,HOUSE_ID,ENROLLID,onset,collect1,collect2,gc_ul1,gc_ul2,home_collected1,home_collected2,mutation,ref,var,season,pcr_result,freq1,freq2,within_host_time,donor_class,generations
1,2,HA,1011,HS1376,MH7755,5033,50141,2014-12-03,2014-12-04,2014-12-05,1.09904707427442e6,9328.66776540432,1,0,HA_A1011G,A,G,2014-2015,A/H3N2,0.0234583211453,0.0,1,Nonsynonymous,4
2,3,HA,1019,HS1516,MH8439,5109,50468,2014-12-21,2014-12-21,2014-12-23,5295.87014994404,13961.6359922664,1,0,HA_G1019A,G,A,2014-2015,A/H3N2,0.0451690779046,0.0,2,Synonymous,8
3,6,HA,140,HS1335,MH7612,5073,50319,2014-11-25,2014-11-25,2014-11-26,224616.057124778,654239.112547579,1,0,HA_A140G,A,G,2014-2015,A/H3N2,0.0264987958706,0.0498634992657,1,Synonymous,4
4,8,HA,1486,HS1381,MH7800,5204,50874,2014-12-05,2014-12-05,2014-12-06,1446.10996315801,21196.4885031859,1,0,HA_A1486T,A,T,2014-2015,A/H3N2,0.103900854115,0.0,1,Nonsynonymous,4
5,9,HA,1670,HS1340,MH7627,5048,50207,2014-11-30,2014-11-30,2014-12-01,4.35098914626686e6,281214.85983637,1,0,HA_G1670A,G,A,2014-2015,A/H3N2,0.0289438607466,0.0349313418453,1,Synonymous,4
6,11,HA,236,HS1376,MH7755,5033,50141,2014-12-03,2014-12-04,2014-12-05,1.09904707427442e6,9328.66776540432,1,0,HA_C236A,C,A,2014-2015,A/H3N2,0.0251913260241,0.0,1,Nonsynonymous,4
7,14,HA,273,HS1416,MH7890,5070,50305,2014-12-05,2014-12-07,2014-12-10,31296.7324317693,93437.9191253739,1,0,HA_C273T,C,T,2014-2015,A/H3N2,0.0511608590998,0.0,3,Synonymous,12
8,16,HA,418,HS1595,MH8925,5234,50993,2015-01-04,2015-01-06,2015-01-09,4.84046142881588e6,34583.56208775,1,0,HA_C418T,C,T,2014-2015,A/H3N2,0.0315552982759,0.0,3,Nonsynonymous,12
9,17,HA,491,HS1566,MH8700,5039,50167,2014-12-27,2014-12-28,2014-12-31,3225.36866342537,20695.5542972638,1,0,HA_T491C,T,C,2014-2015,A/H3N2,0.287317227262,0.0,3,Synonymous,12
10,19,HA,524,HS1390,MH7852,5341,51438,2014-12-07,2014-12-07,2014-12-08,238153.266798938,141146.054111149,1,0,HA_T524C,T,C,2014-2015,A/H3N2,0.0588456364788,0.489384735077,1,Synonymous,4


In [7]:
LL = zeros(100)
#p = Progress(length(LL), 1)
for N in 1:100
    #println(N)
    LL[N]= LogLike(minor,N)
#    next!(p)
end
Nₑ=findfirst(LL,maximum(LL,1)[1])
write_to_summary("Discrete model 6 Ne:",Nₑ)


In [8]:
Nₑ

32

In [9]:
#plot(Nₑ-20:Nₑ+20,LL[Nₑ-20:Nₑ+20],ylim=[LL[Nₑ]-10,LL[Nₑ]+0.5])
#plot(2:10,LL[2:10])

Hmmmm that curve looks a little odd but the result matches our expectation given the diffusion model. It is not smooth because of the binomial nature of the initial model anand our PCR error spread affecting the likelihood of observing certain starting and ending frequencies. 

In [10]:
function CI_interval(LL,Nₑ)

f(x)= x >(LL[Nₑ]-1.92)

Ci = find(f,LL)
low = minimum(Ci,1)[1]
high = maximum(Ci,1)[1]
ci = "$low - $high"
    return(ci)
end

ci = CI_interval(LL,Nₑ)
write_to_summary("Discrete model 6 CI:",ci)
ci

"28 - 41"

## Synonymous

In [11]:
S = minor[minor[:donor_class].=="Synonymous",:]
LL = zeros(100)
#p = Progress(length(LL), 1)
for N in 1:100
    #println(N)
    LL[N]= LogLike(S,N)
#    next!(p)
end
Nₑ=findfirst(LL,maximum(LL,1)[1])
print(Nₑ)
write_to_summary("S Ne:",Nₑ)

37

In [12]:
ci = CI_interval(LL,Nₑ)
write_to_summary("S CI:" ,ci)

In [13]:
#plot(Nₑ-20:Nₑ+20,LL[Nₑ-20:Nₑ+20],ylim=[LL[Nₑ]-20,LL[Nₑ]+0.5])


In [14]:
print(nrow(S))
synom_n = nrow(S)
write_to_summary("S iSNV n:",synom_n)

36

## Nonsynonymous

In [15]:
NS = minor[minor[:donor_class].=="Nonsynonymous",:]
LL = zeros(100)
#p = Progress(length(LL), 1)
for N in 1:100
    #println(N)
    LL[N]= LogLike(NS,N)
#    next!(p)
end
Nₑ=findfirst(LL,maximum(LL,1)[1])
write_to_summary("NS Ne:",Nₑ)
Nₑ

30

In [16]:
ci = CI_interval(LL,Nₑ)
write_to_summary("NS CI:" ,ci)
ci

"21 - 40"

In [17]:
#plot(Nₑ-20:Nₑ+20,LL[Nₑ-20:Nₑ+20],ylim=[LL[Nₑ]-10,LL[Nₑ]+0.5])



In [18]:
print(nrow(NS))
nonsynom_n = nrow(NS)
write_to_summary("NS iSNV n:",nonsynom_n)

27

## Randomly choose 1 iSNV / person

In [19]:
function sub_df(df)
    rows=nrow(df)
    pick = rand(1:rows)
    return(df[pick,:])
end



sub_df (generic function with 1 method)

This is a test that the sub_df function works. 

In [20]:
set = by(minor,:ENROLLID, d-> sub_df(d))
nrow(set)==length(unique(minor[:ENROLLID]))

true

In [21]:
runs = 1000
NₑFits = zeros(runs)
#p = Progress(runs, 1)
maxN = 70
minN = 1
for i in 1:runs
    set = by(minor,:ENROLLID, d-> sub_df(d))
   NₑFits[i] = MLfit(set,minN,maxN)[1]
    if maxN==NₑFits[i]
        while maxN==NₑFits[i]
        println("increasing window")
        maxN +=50
        minN +=50
        NₑFits[i] = MLfit(set,minN,maxN)[1]
        end
    end
#    next!(p)
end

In [22]:
N1per = DataFrame(Ne=NₑFits,iteration = 1:runs)
writetable("../data/processed/secondary/one_per_person.csv",N1per)

write_to_summary("Subset median 6 Ne:",median(NₑFits))

range = quantile(NₑFits,[0.25,0.75]) 

write_to_summary("Subset IQR 6 Ne:",range)

# Sensitivity to outliers


try : p/(1-p) 

In [23]:
ratio(x) = x/(1-x)
minor[:delta2] = -1*abs(ratio.(minor[:freq1]) .- ratio.(minor[:freq2])) ./ minor[:generations] # So the most extreme is on top of the order
minor[:delta] = -1*abs(minor[:freq2] .- minor[:freq1]) ./ minor[:generations] # So the most extreme is on top of the order


minorOrdered = sort!(minor,cols = order(:delta,))


,x,chr,pos,SPECID1,SPECID2,HOUSE_ID,ENROLLID,onset,collect1,collect2,gc_ul1,gc_ul2,home_collected1,home_collected2,mutation,ref,var,season,pcr_result,freq1,freq2,within_host_time,donor_class,generations,delta2,delta
1,81,NS,538,HS1390,MH7852,5341,51438,2014-12-07,2014-12-07,2014-12-08,238153.266798938,141146.054111149,1,0,NS_T538C,T,C,2014-2015,A/H3N2,0.0487008376097,0.48757896261,1,Nonsynonymous,4,-0.22508153994468644,-0.109719531250075
2,72,NR,501,HS1381,MH7800,5204,50874,2014-12-05,2014-12-05,2014-12-06,1446.10996315801,21196.4885031859,1,0,NR_A501G,A,G,2014-2015,A/H3N2,0.0331706001001,0.467637282551,1,Nonsynonymous,4,-0.2110274761681239,-0.10861667061272501
3,19,HA,524,HS1390,MH7852,5341,51438,2014-12-07,2014-12-07,2014-12-08,238153.266798938,141146.054111149,1,0,HA_T524C,T,C,2014-2015,A/H3N2,0.0588456364788,0.489384735077,1,Synonymous,4,-0.22397417822143598,-0.10763477464955
4,124,PB1,1798,HS1381,MH7800,5204,50874,2014-12-05,2014-12-05,2014-12-06,1446.10996315801,21196.4885031859,1,0,PB1_G1798G,G,G,2014-2015,A/H3N2,0.178816730614,0.443234092666,1,Synonymous,4,-0.14458300612173788,-0.066104340513
5,133,PB1,530,HS1381,MH7800,5204,50874,2014-12-05,2014-12-05,2014-12-06,1446.10996315801,21196.4885031859,1,0,PB1_G530A,G,A,2014-2015,A/H3N2,0.25351760292,0.473359560691,1,Nonsynonymous,4,-0.13980310471900476,-0.054960489442750005
6,129,PB1,430,HS1390,MH7852,5341,51438,2014-12-07,2014-12-07,2014-12-08,238153.266798938,141146.054111149,1,0,PB1_G430A,G,A,2014-2015,A/H3N2,0.220772809253,0.0396486125248,1,Synonymous,4,-0.06050931190757523,-0.04528104918205
7,23,HA,638,HS1390,MH7852,5341,51438,2014-12-07,2014-12-07,2014-12-08,238153.266798938,141146.054111149,1,0,HA_G638A,G,A,2014-2015,A/H3N2,0.213904415309,0.0359769154235,1,Synonymous,4,-0.05869759324840944,-0.044481874971375
8,153,PB2,212,HS1381,MH7800,5204,50874,2014-12-05,2014-12-05,2014-12-06,1446.10996315801,21196.4885031859,1,0,PB2_G212A,G,A,2014-2015,A/H3N2,0.302404459968,0.470121801684,1,Nonsynonymous,4,-0.11343269036546312,-0.041929335429
9,120,PB1,1705,HS1381,MH7800,5204,50874,2014-12-05,2014-12-05,2014-12-06,1446.10996315801,21196.4885031859,1,0,PB1_A1705G,A,G,2014-2015,A/H3N2,0.296343011523,0.461350779373,1,Synonymous,4,-0.10883719914934745,-0.0412519419625
10,137,PB1,910,HS1505,MH8391,5254,51073,2014-12-20,2014-12-20,2014-12-22,4.52234282269474e6,21881.0520421267,1,0,PB1_T910A,T,A,2014-2015,A/H3N2,0.340472493385,0.0421432505725,2,Synonymous,8,-0.05902994272949828,-0.0372911553515625


In [24]:
Nₑ = zeros(nrow(minorOrdered))
#p = Progress(nrow(minorOrdered), 1)
maxN = 70
minN = 1
for i in 1:nrow(minorOrdered)
    if maxN>450
        break
    end
    Nₑ[i] = MLfit(minorOrdered[i:nrow(minorOrdered),:],minN,maxN)[1]
    if maxN==Nₑ[i]
        while maxN==Nₑ[i]
        maxN +=50
        minN +=50
        if maxN>450
            break
        end
        Nₑ[i] = MLfit(minorOrdered[i:nrow(minorOrdered),:],minN,maxN)[1]
        end
    end
#    next!(p)
end

In [25]:
print(Nₑ)

[32.0,32.0,40.0,48.0,50.0,51.0,54.0,56.0,57.0,58.0,62.0,65.0,69.0,70.0,77.0,93.0,108.0,110.0,118.0,117.0,114.0,113.0,112.0,118.0,123.0,143.0,149.0,154.0,159.0,152.0,157.0,161.0,153.0,145.0,159.0,162.0,184.0,172.0,161.0,183.0,169.0,155.0,180.0,252.0,240.0,213.0,201.0,349.0,301.0,301.0,301.0,301.0,301.0,301.0,301.0,420.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]

In [26]:
N=Nₑ[Nₑ.>0]
println(N)

x = 0:(length(N)-1) 
println(maximum(x,1))
x = x./nrow(minorOrdered)

println(maximum(x,1))

Ndf = DataFrame(Ne=N,removed = x)
writetable("../data/processed/secondary/removed_data.csv",Ndf)

#plot(x,N,xlim = [0,1],ylim = [0,450])

[32.0,32.0,40.0,48.0,50.0,51.0,54.0,56.0,57.0,58.0,62.0,65.0,69.0,70.0,77.0,93.0,108.0,110.0,118.0,117.0,114.0,113.0,112.0,118.0,123.0,143.0,149.0,154.0,159.0,152.0,157.0,161.0,153.0,145.0,159.0,162.0,184.0,172.0,161.0,183.0,169.0,155.0,180.0,252.0,240.0,213.0,201.0,349.0,301.0,301.0,301.0,301.0,301.0,301.0,301.0,420.0]
[55]
[0.873016]


In [27]:
N[32]
9/63

0.14285714285714285

In [28]:
minorOrdered2 = sort!(minor,cols = order(:delta2,))

Nₑ = zeros(nrow(minorOrdered2))
#p = Progress(nrow(minorOrdered2), 1)
maxN = 70
minN = 1
for i in 1:nrow(minorOrdered2)
    if maxN>450
        break
    end
    Nₑ[i] = MLfit(minorOrdered2[i:nrow(minorOrdered2),:],minN,maxN)[1]
    if maxN==Nₑ[i]
        while maxN==Nₑ[i]
        maxN +=50
        minN +=50
        if maxN>450
            break
        end
        Nₑ[i] = MLfit(minorOrdered2[i:nrow(minorOrdered2),:],minN,maxN)[1]
        end
    end
#    next!(p)
end

print(Nₑ)

N=Nₑ[Nₑ.>0]
println(N)

x = 0:(length(N)-1) 
println(maximum(x,1))
x = x./nrow(minorOrdered2)

println(maximum(x,1))

Ndf = DataFrame(Ne=N,removed = x)
writetable("../data/processed/secondary/removed_data_ratio.csv",Ndf)

#plot(x,N,xlim = [0,1],ylim = [0,450])

[32.0,32.0,39.0,48.0,50.0,51.0,53.0,53.0,54.0,55.0,59.0,63.0,69.0,70.0,82.0,93.0,108.0,110.0,118.0,117.0,114.0,113.0,112.0,129.0,136.0,143.0,149.0,142.0,147.0,152.0,142.0,135.0,139.0,132.0,135.0,147.0,138.0,141.0,159.0,147.0,134.0,123.0,109.0,123.0,154.0,132.0,163.0,150.0,127.0,185.0,151.0,151.0,151.0,151.0,151.0,154.0,218.0,357.0,420.0,0.0,0.0,0.0,0.0][32.0,32.0,39.0,48.0,50.0,51.0,53.0,53.0,54.0,55.0,59.0,63.0,69.0,70.0,82.0,93.0,108.0,110.0,118.0,117.0,114.0,113.0,112.0,129.0,136.0,143.0,149.0,142.0,147.0,152.0,142.0,135.0,139.0,132.0,135.0,147.0,138.0,141.0,159.0,147.0,134.0,123.0,109.0,123.0,154.0,132.0,163.0,150.0,127.0,185.0,151.0,151.0,151.0,151.0,151.0,154.0,218.0,357.0,420.0]
[58]
[0.920635]


This is work exploring why the curve is not monotonic. It is because when we remove some iSNV the likelihood is hurt by the probabilty of hitting the initial frequency which is constrained by the frequency spread in the data.

In [29]:
function LogLikedf(data::DataFrame,Nₑ::Int)
    row,col = size(data)
    
    LL = zeros(row)
    for i in 1:row
        LL[i] = log(pTransition(data[:freq2][i],data[:freq1][i],Nₑ,data[:generations][i]))
    end
    LL
end
minorOrdered[:out_418]=LogLikedf(minorOrdered,418)
minorOrdered[:out_213]=LogLikedf(minorOrdered,)
minorOrdered[:diff] = minorOrdered[:out_213] .-minorOrdered[:out_240]
minorOrdered[45:63,:]

LoadError: MethodError: no method matching LogLikedf(::DataFrames.DataFrame)[0m
Closest candidates are:
  LogLikedf(::DataFrames.DataFrame, [1m[31m::Int64[0m) at In[29]:2[0m

In [30]:
sum(minorOrdered[:out_240][46:63]) - sum(minorOrdered[:out_213][46:63])



LoadError: KeyError: key :out_240 not found

## Simulations 

Ok the goal here is to simulate drift using our starting points and time intervals over a number of different effective populations sizes. We will then estimate the $N\_e$ from those simulations and check our accuracy.

At this point I will round the initial frequency to the nearest available frequency above 0.

This is now run in the separate script in the script directory.


# Generation = 12 hours

In [31]:
minor[:generations] = minor[:within_host_time]*2


LL = zeros(100)
#p = Progress(length(LL), 1)
for N in 1:100
    #println(N)
    LL[N]= LogLike(minor,N)
#    next!(p)
end
Nₑ=findfirst(LL,maximum(LL,1)[1])
#write_to_summary("Discrete model 12 Ne:",Nₑ)

23

In [32]:
#plot(Nₑ-20:Nₑ+20,LL[Nₑ-20:Nₑ+20],ylim=[LL[Nₑ]-10,LL[Nₑ]+0.5])

In [33]:
ci = CI_interval(LL,Nₑ)
write_to_summary("Discrete model 12 CI:",ci)

## Synonymous

In [34]:
S = minor[minor[:donor_class].=="Synonymous",:]
LL = zeros(100)
#p = Progress(length(LL), 1)
for N in 1:100
    #println(N)
    LL[N]= LogLike(S,N)
#    next!(p)
end
Nₑ=findfirst(LL,maximum(LL,1)[1])
print(Nₑ)
write_to_summary("S Ne 12:",Nₑ)

27

In [35]:
ci = CI_interval(LL,Nₑ)
write_to_summary("S CI 12:",ci)

In [36]:
#plot(Nₑ-20:Nₑ+20,LL[Nₑ-20:Nₑ+20],ylim=[LL[Nₑ]-10,LL[Nₑ]+0.5])


In [37]:
print(nrow(S))

36

## Nonsynonymous

In [38]:
NS = minor[minor[:donor_class].=="Nonsynonymous",:]
LL = zeros(100)
#p = Progress(length(LL), 1)
for N in 1:100
    #println(N)
    LL[N]= LogLike(NS,N)
#    next!(p)
end
Nₑ=findfirst(LL,maximum(LL,1)[1])
write_to_summary("NS Ne 12:",Nₑ)

In [39]:
ci = CI_interval(LL,Nₑ)
write_to_summary("NS CI 12",ci)

In [40]:
#plot(LL,ylim=[LL[Nₑ]-10,LL[Nₑ]+0.5])
